In [ ]:
import os
import re
import glob
from tqdm import tqdm

# Specify the folder path containing HTML files
html_folder = '../data/last_trajectory_htmls'

# List of zoom values to set
zoom_values = [17, 18, 20]

# Regex pattern for matching and replacing zoom values
zoom_pattern = re.compile(r'"zoom"\s*:\s*[^,]+,')

# Get all HTML files in the folder
html_files = glob.glob(os.path.join(html_folder, 'trajectory_*.html'))
print(f"Found {len(html_files)} HTML files to process")

# Create progress bar
progress_bar = tqdm(total=len(html_files), desc="Processing HTML files")

# Record success and failure file counts
success_count = 0
error_count = 0

# Iterate through each HTML file
for html_file in html_files:
    try:
        # Extract trajectory ID from filename
        file_name = os.path.basename(html_file)
        traj_id = re.search(r'trajectory_([a-f0-9.]+)\.html', file_name).group(1)

        # Read HTML file content
        with open(html_file, 'r', encoding='utf-8') as f:
            html_content = f.read()

        # Check if original file contains zoom value
        if '"zoom":' not in html_content:
            print(f"Warning: Zoom value not found in file {file_name}, skipping processing")
            error_count += 1
            progress_bar.update(1)
            continue

        # Create and save new file for each zoom value
        for zoom_value in zoom_values:
            # Use regex to replace zoom value
            new_content = zoom_pattern.sub(f'"zoom": {zoom_value},', html_content)

            # Create new filename
            new_file_name = f"trajectory_{traj_id}_{zoom_value}.html"
            new_file_path = os.path.join(html_folder, new_file_name)

            # Save new file
            with open(new_file_path, 'w', encoding='utf-8') as f:
                f.write(new_content)

        # Delete original file
        os.remove(html_file)
        success_count += 1

    except Exception as e:
        print(f"Error processing file {html_file}: {str(e)}")
        error_count += 1

    # Update progress bar
    progress_bar.update(1)

# Close progress bar
progress_bar.close()

# Print processing result statistics
print(f"\n===== Processing Result Statistics =====")
print(f"Total processed: {len(html_files)} files")
print(f"Successfully processed: {success_count} files")
print(f"Processing failed: {error_count} files")
print(f"Total generated: {success_count * len(zoom_values)} new files")

## Batch Modify Zoom Values in HTML Files
This script reads all HTML files in the specified folder, modifies the zoom value in each file to 17, 18, and 20, and generates corresponding new files. Original files will be deleted.

### Operation Flow:
1. Scan all trajectory_*.html files in the specified folder
2. Extract trajectory ID from each file
3. Create a new file for each zoom value (17, 18, 20)
4. Delete original files

### Notes:
- Please ensure to backup original files before running
- Script will display processing progress and result statistics
- If zoom value is not found in a file, it will skip that file and issue a warning